In [79]:
import pandas as pd
import numpy as np
import spacy

import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec

import morfeusz2

import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

# from sentence_transformers import SentenceTransformer
import umap
import hdbscan

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn import utils

In [80]:
df = pd.read_csv('../datasets/ready2use/fake_news_features_tokens.csv', sep=';')

df = df[~df['tokens_str'].isna()]

df['tokens'] = df['tokens_str'].progress_apply(lambda x: x.split(' ') )

100%|███████████████████████████████████| 7759/7759 [00:00<00:00, 266251.09it/s]


In [81]:
df = df[ df['assestment'] != 'brak' ]

Na potrzeby Twojej pracy, wypowiedzi z oceną brak można odrzucić, zbity_zegar interpretować jako fałsz. Resztę danych w zależności od przyjętego modelu, jeśli ocena jest binarna to prawde i blisko_prawdy mozna polaczyc razem, tak samo jak raczej_falsz i falsz. Jeśli jednak ocena jest przedziałem to można pokusić się o przyjęcie wartości 1, 0.75, 0.5, 0.25, 0 zakładając ze 1 to prawda a 0 to pełny fałsz.
sub_title_text_after to komentarz do wypowiedzi, moze pomóc rozwiać wątpliwości co do tego jak oceniane są wartości połówkowe i bliskie pełnych

In [82]:
df['assestment'].value_counts()

Prawda              2817
falsz               1308
Fałsz               1130
zbity_zegar          707
Manipulacja          700
raczej_falsz         288
polprawda            254
Nieweryfikowalne     236
prawda               179
blisko_prawdy         93
Częściowy fałsz        7
Name: assestment, dtype: int64

In [83]:
df.loc[:, 'assestment'] = df['assestment'].replace({
    'falsz' : 'Fałsz',
    'zbity_zegar' : 'Fałsz',
    'raczej_falsz' : 'Fałsz',
    'prawda' : 'Prawda',
    'blisko_prawdy' : 'Prawda',
    'polprawda' : 'Manipulacja',
    'Częściowy fałsz' : 'Manipulacja'
})

In [84]:
df['assestment'].value_counts()

Fałsz               3433
Prawda              3089
Manipulacja          961
Nieweryfikowalne     236
Name: assestment, dtype: int64

In [85]:
df = df[ df['assestment'] != 'Nieweryfikowalne' ]

In [86]:
df = df.reset_index(drop=True)

In [87]:
df['documents'] = [TaggedDocument(doc, [str(i)]) for i, doc in enumerate(df['tokens'].values)]

In [88]:
id_train, id_test = train_test_split(
    df.index.values.tolist(), 
    test_size=0.33, 
    stratify = df['assestment'].values,
    random_state=0)

In [89]:
vec_size = 3

In [90]:
model_dbow = Doc2Vec(dm=0, vector_size=vec_size, negative=5, hs=0, min_count=2, sample = 0, workers=-1)
model_dbow.build_vocab([x for x in tqdm(df['documents'].values[id_train])])

for epoch in tqdm(range(30)):
    model_dbow.train(utils.shuffle([x for x in df['documents'].values[id_train]]), total_examples=len(X_train), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|███████████████████████████████████████████| 30/30 [00:00<00:00, 76.51it/s]


In [91]:
model_dmm = Doc2Vec(dm=1, dm_mean=1, vector_size=vec_size, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)
model_dmm.build_vocab([x for x in tqdm(df['documents'].values[id_train])])

for epoch in tqdm(range(30)):
    model_dmm.train(utils.shuffle([x for x in df['documents'].values[id_train]]), total_examples=len(X_train), epochs=1)
    model_dmm.alpha -= 0.002
    model_dmm.min_alpha = model_dmm.alpha

100%|███████████████████████████████████████████| 30/30 [00:31<00:00,  1.04s/it]


In [92]:
new_model = ConcatenatedDoc2Vec([model_dbow, model_dmm])

In [93]:
df[[ 'e'+str(i) for i in range(vec_size*2) ]] = [
    new_model.infer_vector(d[0]).tolist() for d in df['documents'].values 
]

In [94]:
df.columns

Index(['assestment', 'text_clean', 'sentiment_all', 'sentiment_avg',
       'uniq_words', 'uniq_lemm', 'err', 'net', 'ADJ', 'ADV', 'NOUN',
       'tokens_str', 'tokens', 'documents', 'e0', 'e1', 'e2', 'e3', 'e4',
       'e5'],
      dtype='object')

In [100]:
X_train = df[
    ['sentiment_all', 'sentiment_avg','uniq_words', 'uniq_lemm', 'err', 'net', 
     'ADJ', 'ADV', 'NOUN',
     'e0', 'e1', 'e2', 'e3', 'e4', 'e5']
].values[id_train]

X_test = df[
    ['sentiment_all', 'sentiment_avg','uniq_words', 'uniq_lemm', 'err', 'net', 
     'ADJ', 'ADV', 'NOUN',
     'e0', 'e1', 'e2', 'e3', 'e4', 'e5']
].values[id_test]

y_train = df['assestment'].values[id_train]

y_test = df['assestment'].values[id_test]

In [101]:
X_train.shape

(5013, 15)

In [102]:
X_test.shape

(2470, 15)